In [1]:
!export CUDA_VISIBLE_DEVICES=0

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())


0


In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.device_count()

1

In [4]:
torch.cuda.current_device()

0

In [5]:
# cudaSetDevice(0)
#dataset transcript要在dataset資好夾調
#cuda 在basesolver init調
#dataset要在data.py設定

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())
#! python
# -*- coding: utf-8 -*-
# Author: kun
# @Time: 2019-10-29 20:29

import yaml
import torch
import argparse
import numpy as np

class Para(object):
    a=1

def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda:0')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))
    
#force_cudnn_initialization()
def main():
    # For reproducibility, comment these may speed up training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Arguments
#     parser = argparse.ArgumentParser(description='Training E2E asr.')
#     parser.add_argument('--config', type=str, help='Path to experiment config.')
#     parser.add_argument('--name', default=None, type=str, help='Name for logging.')
#     parser.add_argument('--logdir', default='log/', type=str,
#                         help='Logging path.', required=False)
#     parser.add_argument('--ckpdir', default='ckpt/', type=str,
#                         help='Checkpoint path.', required=False)
#     parser.add_argument('--outdir', default='result/', type=str,
#                         help='Decode output path.', required=False)
#     parser.add_argument('--load', default=None, type=str,
#                         help='Load pre-trained model (for training only)', required=False)
#     parser.add_argument('--seed', default=0, type=int,
#                         help='Random seed for reproducable results.', required=False)
#     parser.add_argument('--cudnn-ctc', action='store_true',
#                         help='Switches CTC backend from torch to cudnn')
#     parser.add_argument('--njobs', default=32, type=int,
#                         help='Number of threads for dataloader/decoding.', required=False)
#     parser.add_argument('--cpu', action='store_true', help='Disable GPU training.')
#     parser.add_argument('--no-pin', action='store_true',
#                         help='Disable pin-memory for dataloader')
#     parser.add_argument('--test', action='store_true', help='Test the model.')
#     parser.add_argument('--no-msg', action='store_true', help='Hide all messages.')
#     parser.add_argument('--lm', action='store_true',
#                         help='Option for training RNNLM.')
#     # Following features in development.
#     parser.add_argument('--amp', action='store_true', help='Option to enable AMP.')
#     parser.add_argument('--reserve-gpu', default=0, type=float,
#                         help='Option to reserve GPU ram for training.')
#     parser.add_argument('--jit', action='store_true',
#                         help='Option for enabling jit in pytorch. (feature in development)')
#     ###
#     paras = parser.parse_args()
    paras = Para()
#     paras.config = './config/aishell_asr_example_lstm4atthead1-test.yaml'
#     paras.name = None
#     paras.logdir = 'log/'
#     paras.ckpdir = 'ckpt/'
#     paras.outdir = 'result/'
#     paras.load = None
#     paras.seed = 0
#     paras.cudnn_ctc = False
#     paras.cpu = False
#     paras.no_pin = False
#     paras.test = True
#     paras.no_msg = False
#     paras.lm = False
#     paras.amp = False
#     paras.reserve_gpu = 0
#     paras.jit = False
    setattr(paras, 'config', './config/cv11Lu_asr_lstm4atthead_allvocab-biclass2.yaml')
    setattr(paras, 'name', None)
    setattr(paras, 'logdir', 'log/')
    setattr(paras, 'ckpdir', 'ckpt/')
    setattr(paras, 'outdir', 'result/')
    setattr(paras, 'load', './ckpt/cv11Lu_asr_lstm4atthead_allvocab_sd0/best_att-VAG01.pth')
    setattr(paras, 'seed', 0)
    setattr(paras, 'cudnn_ctc', False)
    setattr(paras, 'njobs',8)
    setattr(paras, 'cpu', False)
    setattr(paras, 'no_pin', False)
    setattr(paras, 'test', False)
    setattr(paras, 'no_msg', False)
    setattr(paras, 'lm', False)
    setattr(paras, 'amp', False)
    setattr(paras, 'reserve_gpu', 12)
    setattr(paras, 'jit', False)
    setattr(paras, 'gpu', not paras.cpu)
    setattr(paras, 'pin_memory', not paras.no_pin)
    setattr(paras, 'verbose', not paras.no_msg)
    setattr(paras, 'finetune', False)
    setattr(paras, 'binaryClass', True)
    force_cudnn_initialization()

    config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)

    np.random.seed(paras.seed)
    torch.manual_seed(paras.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(paras.seed)

    # Hack to preserve GPU ram just incase OOM later on server
    if paras.gpu and paras.reserve_gpu > 0:
        buff = torch.randn(int(paras.reserve_gpu * 1e9 // 4)).cuda()
        del buff

    if paras.lm:
        # Train RNNLM
        from train_lm import Solver

        mode = 'train'
    else:
        if paras.test:
            # Test ASR
            assert paras.load is None, 'Load option is mutually exclusive to --test'
            from test_asr import Solver

            mode = 'test'
        elif paras.finetune:
            assert paras.load is not None
            from finetune_asr import Solver
            mode = 'train'
        elif paras.binaryClass:
            assert paras.load is not None
            from train_binaryclass2_4folds import Solver
            mode = 'train'
        else:
            # Train ASR
            from train_asr import Solver

            mode = 'train'

    print("\nUsing {} mode\n".format(mode))
    for idx in range(4):
        paras.config = f'./config/cv11Lu_asr_lstm4atthead_allvocab-biclass2-4fold-{idx+1}.yaml'
        config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)
        solver = Solver(config, paras, mode)

        solver.load_data()
    #     solver.print_model()
        solver.set_model()
        solver.exec()
        del solver


0


In [2]:
main()


Using train mode

[INFO] Exp. name : cv11Lu_asr_lstm4atthead_allvocab-biclass2-4fold-1_sd0                                   
[INFO] Loading data... large dataset may took a while.                                                     
Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTT4val-1']
Mozillacv11Dataset CTT4val-1 found wav data: 5
text len: 5
remove None, then wav data: 5, text len: 5
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTT4-4', 'CTT4-2', 'CTT4-3', 'CTT4val-4', 'CTT4val-2', 'CTT4val-3']
Mozillacv11Dataset CTT4-4 found wav data: 19
Mozillacv11Dataset CTT4-2 found wav data: 20
Mozillacv11Dataset CTT4-3 found wav data: 19
Mozillacv11Dataset CTT4val-4 found wav data: 5
Mozillacv11Dataset CTT4val-2 found wav data: 5
Mozillacv11Dataset CTT4val-3 found wav

/home/jupyter-jason5/LAS_Mandarin_PyTorch-master/core/module.py:51: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  feat_len = feat_len // 4


[INFO] Saved checkpoint (step = 795.0K, loss = 0.61) and status @ ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-4fold-1_sd0/best_biclass.pth
43
bestloss3.4014763832092285
tensor(61.0084, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
[INFO] Saved checkpoint (step = 795.1K, loss = 0.58) and status @ ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-4fold-1_sd0/best_biclass.pth
116
bestloss3.3900601863861084
tensor(53.0420, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
tensor(59.5195, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
tensor(53.0600, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
[INFO] Saved checkpoint (step = 795.3K, loss = 0.51) and status @ ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-4fold-1_sd0/best_biclass.pth
335
bestloss3.354658603668213
tensor(52.9038, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
tensor(62.5491, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 73
tensor(52.9993, device='cuda:0', 

text len: 5
remove None, then wav data: 5, text len: 5
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTT4-1', 'CTT4-4', 'CTT4-3', 'CTT4val-1', 'CTT4val-4', 'CTT4val-3']
Mozillacv11Dataset CTT4-1 found wav data: 21
Mozillacv11Dataset CTT4-4 found wav data: 19
Mozillacv11Dataset CTT4-3 found wav data: 19
Mozillacv11Dataset CTT4val-1 found wav data: 5
Mozillacv11Dataset CTT4val-4 found wav data: 5
Mozillacv11Dataset CTT4val-3 found wav data: 5
text len: 74
remove None, then wav data: 74, text len: 74
[INFO] Data spec. | Corpus = vag (from /home/jupyter-jason5/data_process)                                  
[INFO]            | Train sets = ['CTT4-1', 'CTT4-4', 'CTT4-3', 'CTT4val-1', 'CTT4val-4', 'CTT4val-3']	| Number of utts = 74
[INFO]            | Dev sets = ['CTT4val-2']	| Number of utts = 5                                          
[INFO]            | Batch size = 1		| Bucketing = True                                                     
[INFO] I/O spec.  | Audio featu

tensor(0.0001, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 74
tensor(0.0001, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 74
tensor(0.0001, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 74
tensor(0.0001, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 74
tensor(0.0001, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 74
tensor(0.0001, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 74
tensor(0.0001, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 74
tensor(0.0001, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 74
tensor(0.0001, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 74
tensor(0.0001, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 74
tensor(0.0001, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 74
tensor(9.9838e-05, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 74
tensor(9.8050e-05, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 74
tensor(9.6023e-05, device='cud

tensor(55.8360, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
[INFO] Saved checkpoint (step = 795.3K, loss = 0.56) and status @ ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-4fold-3_sd0/best_biclass.pth
335
bestloss3.282402992248535
tensor(55.4550, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
tensor(61.8993, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
tensor(56.5196, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
[INFO] Saved checkpoint (step = 795.6K, loss = 0.75) and status @ ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-4fold-3_sd0/best_biclass.pth
554
bestloss2.72458553314209
tensor(61.0709, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
tensor(56.7233, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
tensor(34.8273, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
tensor(8.2830, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
tensor(6.9796, device='cuda:0', grad_fn=<AddBackward0>) 



text len: 5
remove None, then wav data: 5, text len: 5
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTT4-1', 'CTT4-2', 'CTT4-3', 'CTT4val-1', 'CTT4val-2', 'CTT4val-3']
Mozillacv11Dataset CTT4-1 found wav data: 21
Mozillacv11Dataset CTT4-2 found wav data: 20
Mozillacv11Dataset CTT4-3 found wav data: 19
Mozillacv11Dataset CTT4val-1 found wav data: 5
Mozillacv11Dataset CTT4val-2 found wav data: 5
Mozillacv11Dataset CTT4val-3 found wav data: 5
text len: 75
remove None, then wav data: 75, text len: 75
[INFO] Data spec. | Corpus = vag (from /home/jupyter-jason5/data_process)                                  
[INFO]            | Train sets = ['CTT4-1', 'CTT4-2', 'CTT4-3', 'CTT4val-1', 'CTT4val-2', 'CTT4val-3']	| Number of utts = 75
[INFO]            | Dev sets = ['CTT4val-4']	| Number of utts = 5                                          
[INFO]            | Batch size = 1		| Bucketing = True                                                     
[INFO] I/O spec.  | Audio featu

tensor(8.9646e-05, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
tensor(8.7381e-05, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
tensor(8.5533e-05, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
tensor(8.3149e-05, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
tensor(8.1599e-05, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
tensor(7.9572e-05, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
tensor(7.8023e-05, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
tensor(7.6115e-05, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
tensor(7.4506e-05, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
tensor(7.2301e-05, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
tensor(7.1049e-05, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
tensor(7.0155e-05, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
[INFO] Saved checkpoint (step = 799.6K, loss = 0.00) and status 

tensor(3.9458e-05, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 75
[INFO] Saved checkpoint (step = 802.5K, loss = 0.00) and status @ ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-4fold-4_sd0/best_biclass.pth
7489
bestloss0.00829895306378603
tensor(3.9220e-05, device='cuda:0', grad_fn=<AddBackward0>) 

802500


In [ ]:
print('asd')

In [11]:
!python eval.py --file ~/LAS_Mandarin_PyTorch-master/result/mozillacv11_asr_stm4atthead-test_test_output.csv


============  Result of /home/jupyter-jason3/LAS_Mandarin_PyTorch-master/result/mozillacv11_asr_stm4atthead-test_test_output.csv ============
 -----------------------------------------------------------------------
| Statics		|  Truth	|  Prediction	| Abs. Diff.	|
 -----------------------------------------------------------------------
| Avg. # of chars	|  8.01	|  8.10	|  0.16		|
| Avg. # of words	|  1.00	|  1.00	|  0.00		|
 -----------------------------------------------------------------------
 ---------------------------------------------------------------
| Error Rate (%)| Mean		| Std.		| Min./Max.	|
 ---------------------------------------------------------------
| Character	| 7.9479	| 21.68		| 0.00/240.00	|
| Word		| 18.1725	| 38.56		| 0.00/100.00	|
 ---------------------------------------------------------------
Note : If the text unit is phoneme, WER = PER and CER is meaningless.

